# Side Effects: References and Mutable Vars in Lettuce

In this lecture, we will examine how we can augment Lettuce with side effects. Thus far, lettuce has
been a _pure functional language_. What is purity? In programming, _purity_ refers to a lack of _side effects_.
Side effects refer to actions of a program that have a "global effect" on the computation.  Examples of side effects include:
- Mutable variables eg., var in scala that can be assigned.  The assignment of var has a side effect, since it can modify the effect of statements before and after. 
Let us see an example that illustrates this side effect in Scala.

In [1]:
class SideEffectClass(var x: Int) {
    // get simply returns the value of x
    def get(): Int = x
    // mut increments x by 1 --> this function has a side effect.
    // executing mut affects the future behavior of the get function
    def mut(): Unit = { x = x + 1}
}

val m1 = new SideEffectClass(10)

println(m1.get())

m1.mut()

println(m1.get())

10
11


defined class SideEffectClass
m1: SideEffectClass = ammonite.$sess.cmd0$Helper$SideEffectClass@127a979d

Notice that the function _mut_ has a side effect. When executed, it actually affects the future behavior of the _get_ function. This kind of a "non-local" effect is called a _side effect_.

- Printing, reading from a file, sending a packet to a network, acquiring a lock, .. are all operations that have a side effect. They produce an effect that is observable by the programmer. And thus, we can imagine what the user observes after a print statement to be different from what they observe before a print statement.

### Are side effects bad?

Although the term side effects has a medical connotation that is not desirable, side effects are neither good nor bad. For instance, if the purpose of a program is to maintain a client-server interaction with the user, it is impossible to avoid side effects. However, side effects could make a program harder to comprehend. Often, undocumented side effects frustrate users of libraries who may not have access to the code. Programmers like __modularity__, wherein a program is broken into independent modules that build important parts of the functionality of the program. The program itself is put together by composing these modules. However, side effects make modularity difficult. 

~~~
Engineer A is building code to collect some data from the user and engineer B is building code to process it. Let us say that as a result of calling a function written by A, many files are opened under the hood but not closed. It is possible then that B may see that her application is crashing due to out of disk space errors. 
This is caused by opening files on the disk, which is a side effect.
This does not mean that the side effect caused by A's code is avoidable. But it must be understood and documented carefully, so that people who use the code can be aware of it, expect it or deal with it.
~~~
 
 
## Adding Mutable References to Lettuce

We will now discuss an extension of lettuce with mutable references. What are mutable references? 

- Mutable references are addresses to locations in memory that hold a value.  
  - For instance, consider memory cell 1 holds the value `NumValue(1.0)`.
  - A mutable reference is of the form `Ref(1)`, that tells us that this is a reference to memory address `1` and looking up address 1, we find the value `NumValue(1.0)` stored there.

- We will have an abstraction of memory wherein, the memory has cells numbered 0, 1, 2, ... and each cell can hold a value. This could be a number, a closure, a boolean or another reference (reference to reference!), etc..

We will augment Lettuce with three statements:
- $NewRef(\text{value})$, create a new memory cell and place `value` inside it. This returns a reference to the memory cell created.
- $DeRef(\text{reference})$, fetch the value stored in the memory cell that is referenced by `reference`.
- $AssignRef(\text{reference}, \text{value})$, change the memory cell pointed to by `reference` to have the value `value`. 

References are very much like pointers in C/C++. However, the main difference is that pointers are very low level. Here our references can store arbitrary (boxed) values inside them. This includes all the values we have encountered thus far: numbers, booleans, closures and references themselves. Therefore, unlike pointers, the size of the memory being pointed to can be variable.

### Step 1: Grammar of Lettuce with References

Let us take a stripped down version of Lettuce grammar with function calls. Let us leave recursion out of it although we can easily handle recursions as well.


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{NewRef}(\mathbf{Expr}) & \text{create a new reference and place the value of expr in it.}\\
 & | & \color{red}{DeRef}(\mathbf{Expr}) & \text{dereference a reference and return the value}\\
 & | & \color{red}{AssignRef}(\mathbf{Expr}, \mathbf{Expr}) & \text{assign a reference to a value. }
\end{array}$$


Some notes
- `NewRef` takes in an expression and makes a fresh reference. It them stores the value into this reference.
- `DeRef` takes in an expr but as you will see, the expr needs to be a reference or else we will throw an error and bail.
- `AssignRef` assigns a reference produced by evaluating its first arugment to the value from the second argument. What should assignment return? This depends on our design:
  - We can make AssignRef return nothing and the name for nothing is called "Unit" (also called void in languages like C/C++)
  - We can make AssignRef evaluate to the actual value that is being assigned. This is common in languages like C/C++
  
For example in C, we can write
~~~
  y = x = 10 + 15
~~~

The idea here is that `x = 10 + 25` evaluates to `25` in addition to having the side effect of assigning `25` to `x`. And thus, the nested assignment assigns `25` to `y`.

We can use `AssignRef` to say the same in Lettuce.

All of this can be written up in Scala to form our AST.

In [2]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// New Ref
case class NewRef(e: Expr) extends Expr

//DeRef
case class DeRef(lval: Expr) extends Expr

//AssignRef
case class AssignRef(lval: Expr, rval: Expr) extends Expr


defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef

## Examples: Programs with References

Let us go over some programs in Lettuce with references to get our intuitions set.

~~~
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
   z
~~~

What is the value returned by the program above? It creates a new cell with value 10 and stores the
reference to the cell in x. Next, it dereferences x, and adds 1 to it storing the result in y and
finally, it assigns y to x, placing the assigned value inside z. We return z.

The value computed should be 11.

To understand the effect of this program, when we run it, the following steps happen:
- x gets assigned to a new cell say cell number 0 in memory. cell number 0 stores value 10. x gets bound to a reference to cell number 0.
- cell number 0 gets looked up (deref) yielding the value 10. Thus, y gets bound to  the value 11.
- the assignment overwrites cell number 0 in memory with the value 11. The return value of the assignment is
the value 11 that gets bound to z.
- the value of z is returned.

Let us try a more complex example:

~~~
let incr = function (x) 
           AssignRef(x, DeRef(x) + 1)
in 
let decr = function (x) 
           AssignRef(x, DeRef(x) - 1)
in 
let zero = function (x) 
          AssignRef(x, 0)
in 
let r = NewRef(15) in 
let dummy0 = zero(r) in 
let dummy1 = incr(r) in 
let dummy2 = incr(r) in
let dummy3 = decr(r) in 
  dummy3 
~~~


## Semantics with References

Now we are ready to talk about semantics of Lettuce with explicit references. There are two big changes we need to make.

1. We need to make a new value type for references since our expressions an evaluate to real numbers, booleans, closures and now <font color="red"> references </font>.
2. To go hand in hand with references, we need to define an abstract notion of memory. We will call this a <font color="red"> store </font>.


### Stores

Let us start with a store. We will assume that memory is "unlimited" (and yes, the other leg has bells on!). 
Memory address are going to be numbered 0, 1, 2, .... with natural numbers and each address is going to be associated with a value. 

The store needs to support the following operations.

- Create a new memory cell in the store and assign it to a value. This is exactly what will implement the `NewRef` operation. Let us call it <font color="red">createNewCell</font> operation on stores.

- Lookup the value of a memory cell. Let us call it <font color="red"> lookupCellValue </font> operation. If the value does not exist, we will return error (and bail).

- Assign a cell to a new value. Let us call it <font color="red"> assignToCell </font>

Hand in hand with stores, we have to extend our value type. Existing value types are
- `ErrorValue` to denote error -- though in our physical implementation, we have never issued this value. We rather prefer to bail with an exception. We will continue to do so to make our lives simpler. Operational semantics will be written with error values, but the actual code will bail on error.

- `NumValue(f)` for number `f`. We will denote these as reals $\mathbb{R}$ in our semantics.
- `BoolValue(b)` for boolean `b`. We will denote these as $\mathbb{B}$ in our semantics.
- `Closure(x, e, sigma)` for closures. We will denote these as $\mathbf{Closure}$ in our semantics.

Finally, we add references the new kid on the block:
- `Reference(j)`, which is a reference to cell number j in our store. 

### Implementation of Store and Values (with references)

Let us implement store: there are many ways of doing this.  Since we are gluttons for punishment, and more seriously, since we want to illustrate how our semantics are going to translate into the interpreter, we are going
to model immutable stores. In other words, every time, we create a new cell or reassign a cell, we are going to make an entirely new store. 

In [3]:
sealed trait Value


/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store */

case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    



defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell

## Operational Semantics with Side Effects.

Thus far, we have looked at operational semantics for pure lettuce. Now that we have added side effects (create new cells, change the value of cells), it is important to write our operational semantics.

The most important change is this
- Evaluation of an expression used to have two arguments: the expression and the environment. Now, evaluation will have three arguments: the expression, the env and the store.
- Previously, evaluation gave one result: just the value that arose from evaluating the expression. Now, we will have to capture two things: the value and the new store.

Thus, our eval function will have the form:

$$ \mathbf{eval}(\texttt{e}, \text{env}, \text{store}) = (\text{value}, \text{new-store}) $$


Things become slightly more complicated. But keep reciting "side effects", "side effects" "side effects", ... and all will be clear (at least less murky :-)



### Semantic Rules With Side Effects

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array}\ \ \text{(#3)}} $$
$$\newcommand\eval{\mathbf{eval}}$$
Let us start with the familiar ones.

Evaluating a constant under a env. $\sigma$ and store $s$ results in the value of constant and the same store unchanged. 
$$\semRule{}{\eval(\texttt{Const(f)}, \sigma, s) = ( f, s) }{const}$$

Evaluating an identifier also remains unchanged.
$$\semRule{x \in \text{domain}(\sigma)}{\eval(\texttt{Ident(x)}, \sigma, s) = (\sigma(x), s)}{ident-ok}\;\;\;
 \semRule{x \not\in \text{domain}(\sigma)}{\eval(\texttt{Ident(x)}, \sigma, s) = (\mathbf{error}, s)}{ident-nok}\;\;\; $$
 
The rule for Plus, Minus, Mult will become more complicated: look at the rule very carefully for Plus. 

$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; v_1 \in \mathbb{R}\;\;\; \eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2})\;\;\; v_2 \in \mathbb{R} }{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (v_1 + v_2 , \color{blue}{s_2})}{plus-ok} $$

Let us read it out: 
- Evaluating $\texttt{e1}$ under env. $\sigma$ and store $s$ yields a real number $v_1$ with new store $\color{red}{s_1}$.
- To evaluate the second sub-expression $\texttt{e2}$, we need to pass the store $\color{red}{s_1}$ as input, because it is possible to have a side effect when evaluating $\texttt{e1}$ that could have updated the store.
- The overall rule specifies the left to right evaluation.

We will need to write error handling rules. Here we can resort to short circuiting as before.
$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; \color{red}{v_1 \not\in \mathbb{R}}}{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (\mathbf{error} , \color{red}{s_1})}{plus-nok-1} \;\;
\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; v_1 \in \mathbb{R}\;\;\; \eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2})\;\;\; \color{red}{v_2 \not\in \mathbb{R}} }{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (\mathbf{error} , \color{blue}{s_2})}{plus-nok-2}
$$

Why is evaluation order of arithmetic operators important? Let us take an example program.

~~~
let f = function (x)
          let tmp = DeRef(x) in 
          let dummy = assignRef(x, 25) in (*- watch out, side effect -*)
               tmp 
    in 
let r = NewRef(10) in 
   f(r) + f(r)
~~~

The function $f$ takes in a reference $x$ and returns the derefrence of $x$ but at the same time, it
assigns $x$ to $25$. The next time, $f(r)$ is called, we have to model this side effect. As you can see, the
order of evaluation may matter in these case.  For instance take the same program changing the `Plus` to a `Minus`.

~~~
let f = function (x)
          let tmp = DeRef(x) in 
          let dummy = assignRef(x, 25) in (*- watch out, side effect -*)
               tmp 
    in 
let r = NewRef(10) in 
   f(r) - f(r)
~~~

It matters if we evaluate from left to right or right to left. Typical evaluation order is left to right for Plus, Minus, Mult, Div and so on.

#### Exercise: Write the semantics for Minus, Mult, Geq, Eq. They are going to be very similar to Plus.

Now, we can write the semantics for `IfThenElse` which is also very similar.

$$\semRule{\eval(\texttt{e}, \sigma, s) = (true, \color{red}{s_1})\;\;\; 
\eval(\texttt{e1}, \sigma, \color{red}{s_1}) = (v_1, \color{blue}{s_2}) }{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (v_1, \color{blue}{s_2})}{ite-then}$$


$$\semRule{\eval(\texttt{e}, \sigma, s) = (false, \color{red}{s_1})\;\;\;
\eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2}) }{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (v_2, \color{blue}{s_2})}{ite-else}$$

We can also write an error case if evaluating the condition yields a non boolean value (could also be error).
$$\semRule{\eval(\texttt{e}, \sigma, s) = (b, \color{red}{s_1})\;\;\; b \not\in \mathbb{B}}{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (\mathbf{error}, \color{red}{s_1})}{ite-nok}$$

### Let Binding

The rule for let bindings is as

$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s'})\;\; v_1 \not= \mathbf{error} }{ \eval(\texttt{Let(x, e1, e2)}, \sigma, s) = \eval(\texttt{e2}, \sigma[x \mapsto v_1] , \color{red}{s'}) }{let-ok}$$

As an exercise, you should complete the Let binding.

### Function Calls.

The handling of function calls is very similar as well. A function definition has no effect on the store.

$$\semRule{}{\eval(\texttt{FunDef(x, e)}, \sigma, s) = (\texttt{Closure}(x, e, \sigma), s )}{fundef}$$

Function calls are tricky. Suppose we have a function call expression

~~~
 <function-expr> (arg-expr) 
~~~

We will need to decide which one to evaluate first, should we eval the `function-expr` to find out what function is being called first and then the `arg-expr` or vice-versa? Previously it did not matter and now it does. However, programmers in the western world are very used to reading programs as they do natural language: from left to right. We will adopt the same convention.

$$\semRule{\eval(\texttt{e1}, \sigma, s) = (\texttt{Closure}(x, \texttt{fbody}, \pi), s_1) \;\;\; \eval(\texttt{e2}, \sigma, s_1) = (v, s_2),\;\; v \not= \mathbf{error} }{
\eval(\texttt{FunCall(e1, e2)}, \sigma, s) = \eval(\texttt{fbody}, \pi[x \mapsto v], s_2) }{funcall-ok}$$

Notice how the body of the closure is being evaluated using the store that comes after the arguments have been evaluated.

### Operations on References

Finally, we are ready to write down operations over references themselves.

Let us start with NewRef

$$\semRule{\eval(\texttt{e}, \sigma, s) = (v, s_1), \;\; v \not= \mathbf{error},\;\;\; \texttt{createNewCell}(s_1,v) = (j, s_2) }{\eval(\texttt{NewRef(e)}, \sigma, s) = (\texttt{Reference}(j), s_2) }{newref-ok}$$

The idea here is stated as follows: In order to evaluate an expression of the form `NewRef(e)`, under env. $\sigma$ and current store $s$.
- First evaluate $e$ under current environment and store, to create a new store $s_1$. Let $v$ be the value.
- If the value is not error, then create a new cell on the store $s_1$. Let $j$ be the address of the cell created and $s_2$ be the store that includes all old cells and the newly minted cell $j$.
- The value of `NewRef(e)` is a reference to the newly created cell `j` with the resulting store being $s_2$.

Next, let us do DeRef. 

$$\semRule{\eval(\texttt{e}, \sigma, s) = (r, s_1), \;\; r = \texttt{Reference}(j)\;\; \texttt{lookupCell}(s_1, j) = v }{\eval(\texttt{Deref(e)}, \sigma, s) = (v, s_1) }{deref-ok}$$

Finally, we can do assignment


$$\semRule{\eval(\texttt{e1}, \sigma, s) = (r, s_1), \;\; r = \texttt{Reference}(j)\;\; \eval(\texttt{e2}, \sigma, s_1) = (v_2, s_2), \;\; \texttt{assignToCell}(s_2, j, v_2) = s_3 }{\eval(\texttt{AssignRef(e1, e2)}, \sigma, s) = (v_2, s_3) }{assign-ref-ok}$$

**Exercise** Write down some of the error cases for the NewRef, DeRef and AssignRef operations. 

We are ready to write an interpreter.

In [4]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) 
                (env(x), store)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        case NewRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            val (store2, j) = createNewCell(store1, v)
            (Reference(j), store2)
        }
        
        case DeRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val v = lookupCellValue(store1, j)
                    (v, store1)
                }
                case _ => throw new IllegalArgumentException(s"Deref applied to an expression that does not evaluate to a reference")
            }
        }
        
        case AssignRef(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case Reference(j) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    val store3 = assignToCell(store2, j, v2)
                    (v2, store3)// Return the  assigned value
                }
                case _ => throw new IllegalArgumentException(s"AssignRef applied to argument that is not a reference")
                
            }
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    


defined function evalExpr
defined function evalProgram

In [5]:
/*
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
   z*/

val z = Ident("z")
val x = Ident("x")
val y = Ident("y")

val e1 = Let("z", AssignRef(x, y), z)
val e2 = Let("y", Plus(Const(1.0), DeRef(x)), e1)
val e = Let("x", NewRef(Const(10.0)), e2)
val prog = TopLevel(e)

println(s"Result: ${evalProgram(prog)}")

Result: NumValue(11.0)


z: Ident = Ident("z")
x: Ident = Ident("x")
y: Ident = Ident("y")
e1: Let = Let("z", AssignRef(Ident("x"), Ident("y")), Ident("z"))
e2: Let = Let(
  "y",
  Plus(Const(1.0), DeRef(Ident("x"))),
  Let("z", AssignRef(Ident("x"), Ident("y")), Ident("z"))
)
e: Let = Let(
  "x",
  NewRef(Const(10.0)),
  Let(
    "y",
    Plus(Const(1.0), DeRef(Ident("x"))),
    Let("z", AssignRef(Ident("x"), Ident("y")), Ident("z"))
  )
)
prog: TopLevel = TopLevel(
  Let(
    "x",
    NewRef(Const(10.0)),
    Let(
      "y",
      Plus(Const(1.0), DeRef(Ident("x"))),
      Let("z", AssignRef(Ident("x"), Ident("y")), Ident("z"))
    )
  )
)

In [6]:
/*
let incr = function (x) 
           AssignRef(x, DeRef(x) + 1)
in 
let decr = function (x) 
           AssignRef(x, DeRef(x) - 1)
in 
let zero = function (x) 
          AssignRef(x, 0)
in 
let r = NewRef(15) in 
let dummy0 = zero(r) in 
let dummy1 = incr(r) in 
let dummy2 = incr(r) in
let dummy3 = decr(r) in 
  dummy3 
  */

val incr = Ident("incr")
val decr = Ident("decr")
val zero = Ident("zero")
val r = Ident("r")
val x = Ident("x")
val e1 = Let("dummy3", FunCall(decr, r), Ident("dummy3"))
val e2 = Let("dummy2", FunCall(incr, r),  e1)
val e3 = Let("dummy1", FunCall(incr, r), e2)
val e4 = Let("dummy0", FunCall(zero, r), e3)
val rr = Let("r", NewRef(Const(15.0)), e4)
val z = Let("zero", FunDef("x", AssignRef(x, Const(0.0))), rr)
val d = Let("decr", FunDef("x", AssignRef(x, Minus(DeRef(x), Const(1.0)))), z)
val i = Let("incr",FunDef("x", AssignRef(x, Plus(DeRef(x), Const(1.0)))), d )
val prog = TopLevel(i)

println(s"Result: ${evalProgram(prog)}")


Result: NumValue(1.0)


incr: Ident = Ident("incr")
decr: Ident = Ident("decr")
zero: Ident = Ident("zero")
r: Ident = Ident("r")
x: Ident = Ident("x")
e1: Let = Let("dummy3", FunCall(Ident("decr"), Ident("r")), Ident("dummy3"))
e2: Let = Let(
  "dummy2",
  FunCall(Ident("incr"), Ident("r")),
  Let("dummy3", FunCall(Ident("decr"), Ident("r")), Ident("dummy3"))
)
e3: Let = Let(
  "dummy1",
  FunCall(Ident("incr"), Ident("r")),
  Let(
    "dummy2",
    FunCall(Ident("incr"), Ident("r")),
    Let("dummy3", FunCall(Ident("decr"), Ident("r")), Ident("dummy3"))
  )
)
e4: Let = Let(
  "dummy0",
  FunCall(Ident("zero"), Ident("r")),
  Let(
    "dummy1",
    FunCall(Ident("incr"), Ident("r")),
    Let(
      "dummy2",
      FunCall(Ident("incr"), Ident("r")),
      Let("dummy3", FunCall(Ident("decr"), Ident("r")), Ident("dummy3"))
    )
  )
)
rr: Let = Let(
  "r",
  NewRef(Const(15.0)),
  Let(
    "dummy0",
    FunCall(Ident("zero"), Ident("r")),
    Let(
      "dummy1",
      FunCall(Ident("incr"), Ident("r")),
     

In [7]:
/* 
let f = function (x)
          let tmp = DeRef(x) in 
          let dummy = assignRef(x, 25) in (*- watch out, side effect -*)
               tmp 
    in 
let r = NewRef(10) in 
   f(r) - f(r)
   */

val r = Ident("r")
val x = Ident("x")
val f = Ident("f")

val e1 = Minus(FunCall(f, r), FunCall(f, r))
val e2 = Let("r", NewRef(Const(10.0)), e1)
val fbody = Let("tmp", DeRef(x), Let("dummy", AssignRef(x, Const(25.0)), Ident("tmp")))
val fdef = FunDef("x", fbody)
val e3 = Let("f", fdef, e2)
val prog = TopLevel(e3)


println(s"Result: ${evalProgram(prog)}")


Result: NumValue(-15.0)


r: Ident = Ident("r")
x: Ident = Ident("x")
f: Ident = Ident("f")
e1: Minus = Minus(
  FunCall(Ident("f"), Ident("r")),
  FunCall(Ident("f"), Ident("r"))
)
e2: Let = Let(
  "r",
  NewRef(Const(10.0)),
  Minus(FunCall(Ident("f"), Ident("r")), FunCall(Ident("f"), Ident("r")))
)
fbody: Let = Let(
  "tmp",
  DeRef(Ident("x")),
  Let("dummy", AssignRef(Ident("x"), Const(25.0)), Ident("tmp"))
)
fdef: FunDef = FunDef(
  "x",
  Let(
    "tmp",
    DeRef(Ident("x")),
    Let("dummy", AssignRef(Ident("x"), Const(25.0)), Ident("tmp"))
  )
)
e3: Let = Let(
  "f",
  FunDef(
    "x",
    Let(
      "tmp",
      DeRef(Ident("x")),
      Let("dummy", AssignRef(Ident("x"), Const(25.0)), Ident("tmp"))
    )
  ),
  Let(
    "r",
    NewRef(Const(10.0)),
    Minus(FunCall(Ident("f"), Ident("r")), FunCall(Ident("f"), Ident("r")))
  )
)
prog: TopLevel = TopLevel(
  Let(
    "f",
    FunDef(
      "x",
      Let(
        "tmp",
        DeRef(Ident("x")),
        Let("dummy", AssignRef(Ident("x"), Const(25.0)

In [8]:
/*
let x = NewRef(10) in 
   let g = function (y) 
            DeRef(x)
        in 
       let dummy = AssignRef(x, 20) in 
            g (dummy)
*/
val dummy = Ident("dummy")
val x = Ident("x")
val g = Ident("g")
val e1 = FunCall(g, dummy)
val e2 = Let("dummy", AssignRef(x, Const(20)), e1)
val fdef = FunDef("y", DeRef(x))
val e3 = Let("g", fdef, e2)
val e4 = Let("x", NewRef(Const(10)), e3)
val prog = TopLevel(e4)


println(s"Result: ${evalProgram(prog)}")


Result: NumValue(20.0)


dummy: Ident = Ident("dummy")
x: Ident = Ident("x")
g: Ident = Ident("g")
e1: FunCall = FunCall(Ident("g"), Ident("dummy"))
e2: Let = Let(
  "dummy",
  AssignRef(Ident("x"), Const(20.0)),
  FunCall(Ident("g"), Ident("dummy"))
)
fdef: FunDef = FunDef("y", DeRef(Ident("x")))
e3: Let = Let(
  "g",
  FunDef("y", DeRef(Ident("x"))),
  Let(
    "dummy",
    AssignRef(Ident("x"), Const(20.0)),
    FunCall(Ident("g"), Ident("dummy"))
  )
)
e4: Let = Let(
  "x",
  NewRef(Const(10.0)),
  Let(
    "g",
    FunDef("y", DeRef(Ident("x"))),
    Let(
      "dummy",
      AssignRef(Ident("x"), Const(20.0)),
      FunCall(Ident("g"), Ident("dummy"))
    )
  )
)
prog: TopLevel = TopLevel(
  Let(
    "x",
    NewRef(Const(10.0)),
    Let(
      "g",
      FunDef("y", DeRef(Ident("x"))),
      Let(
        "dummy",
        AssignRef(Ident("x"), Const(20.0)),
        FunCall(Ident("g"), Ident("dummy"))
      )
    )
  )
)

In [9]:
var x = 10
var y = x
y = 15
println(x)
println(y)

10
15


x: Int = 10
y: Int = 15

In [10]:
print(x)

10

In [10]:
let x = newref(10) in //Reference(0)
    let y = x in // y |-> Reference(0)
      let dummy = assign(y, 15) in 
          deref(x)

cmd10.sc:1: not found: value let
val res10 = let x = newref(10) in //Reference(0)
            ^cmd10.sc:1: postfix operator x needs to be enabled
by making the implicit value scala.language.postfixOps visible.
----
This can be achieved by adding the import clause 'import scala.language.postfixOps'
or by setting the compiler option -language:postfixOps.
See the Scaladoc for value scala.language.postfixOps for a discussion
why the feature needs to be explicitly enabled.
val res10 = let x = newref(10) in //Reference(0)
                ^Compilation Failed

: 